In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op

from mod import load_data, alt_theme

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [4]:
# Parameters
renderer = "kaggle"
start_date = "2019-01-01"
end_date = "2019-02-01"


In [5]:
comments, issues, prs = load_data('../data/')
bot_names = pd.read_csv('bot_names.csv')['names'].tolist()
comments = comments.query('author not in @bot_names')
issues = issues.query('author not in @bot_names')
prs = prs.query('author not in @bot_names')

/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  comments = pd.concat(comments)


/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  issues = pd.concat(issues)
/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  prs = pd.concat(prs)


In [6]:
# Only keep the dates we want
comments = comments.query('updatedAt > @start_date and updatedAt < @end_date')
issues = issues.query('updatedAt > @start_date and updatedAt < @end_date')
prs = prs.query('updatedAt > @start_date and updatedAt < @end_date')

In [7]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [8]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [9]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [10]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/nteract](https://github.com/nteract)

In [11]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-01-01** to **2019-02-01**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [12]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 31 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [13]:
merged = prs.query('state == "MERGED" and closedAt > @start_date and closedAt < @end_date')

In [14]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                         repo  author
  47          nteract                    papermill      13
  44          nteract                     hydrogen       4
  42          nteract                         docs       3
  41          nteract                     commuter       2
  43          nteract                     examples       2
  45          nteract                          nes       1
  46          nteract                   nteract.io       1
  34       jupyterlab                   jupyterlab     160
  35       jupyterlab        jupyterlab-commenting      25
  33       jupyterlab            jupyter-renderers      10
  40       jupyterlab               jupyterlab-toc      10
  36       jupyterlab      jupyterlab-google-drive       5
  37       jupyterlab             jupyterlab-latex       5
  38       jupyterlab  jupyterlab-metadata-service       5
  39       jupyterlab        jupyterlab-shortcutui       5
  23       jupyterhub         jupyter-server-proxy      34
  32       jupyterhub       zero-to-jupyterhub-k8s      34
  27       jupyterhub          nativeauthenticator      32
  18       jupyterhub                    binderhub      22
  26       jupyterhub          mybinder.org-deploy      20
  29       jupyterhub      the-littlest-jupyterhub      16
  22       jupyterhub       jupyter-rsession-proxy      14
  30       jupyterhub                traefik-proxy      12
  24       jupyterhub                   jupyterhub      10
  28       jupyterhub                 team-compass       8
  31       jupyterhub                  yarnspawner       8
  16       jupyterhub         alabaster-jupyterhub       4
  17       jupyterhub                       binder       2
  19       jupyterhub      configurable-http-proxy       2
  20       jupyterhub        firstuseauthenticator       2
  21       jupyterhub                   helm-chart       2
  25       jupyterhub                  kubespawner       2
  15  jupyter-widgets                   ipyleaflet      32
  3           jupyter           enterprise_gateway      17
  5           jupyter                 jupyter-book      13
  13          jupyter                  repo2docker      10
  12          jupyter                     notebook       7
  2           jupyter                docker-stacks       6
  11          jupyter                     nbviewer       5
  7           jupyter                     jvm-repr       4
  6           jupyter            jupyter.github.io       3
  8           jupyter                    nbconvert       3
  4           jupyter                      jupyter       2
  10          jupyter                     nbgrader       2
  1           jupyter   cookiecutter-docker-stacks       1
  9           jupyter           nbconvert-examples       1
  14          jupyter                      surveys       1
  0           ipython                      ipython      27,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['papermill', 'hydrogen', 'docs', 'commuter', 'examples', 'nes', 'nteract.io', 'jupyterlab', 'jupyterlab-commenting', 'jupyter-renderers', 'jupyterlab-toc', 'jupyterlab-google-drive', 'jupyterlab-latex', 'jupyterlab-metadata-service', 'jupyterlab-shortcutui', 'jupyter-server-proxy', 'zero-to-jupyterhub-k8s', 'nativeauthenticator', 'binderhub', 'mybinder.org-deploy', 'the-littlest-jupyterhub', 'jupyter-rsession-proxy', 'traefik-proxy', 'jupyterhub', 'team-compass', 'yarnspawner', 'alabaster-jupyterhub', 'binder', 'configurable-http-proxy', 'firstuseauthenticator', 'helm-chart', 'kubespawner', 'ipyleaflet', 'enterprise_gateway', 'jupyter-book', 'repo2docker', 'notebook', 'docker-stacks', 'nbviewer', 'jvm-repr', 'jupyter.github.io', 'nbconvert', 'jupyter', 'nbgrader', 'cookiecutter-docker-stacks', 'nbconvert-examples', 'surveys', 'ipython'],
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [15]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [16]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [17]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:     username  authored authorAssociation
    0    BoboTiG         6       CONTRIBUTOR
    1     SSteve         3       CONTRIBUTOR
    2   gpotter2         3       CONTRIBUTOR
    3      ibell         3       CONTRIBUTOR
    4    sanyam5         3       CONTRIBUTOR
    5  techtonik         3       CONTRIBUTOR
    6     tomyun         3       CONTRIBUTOR
    7   tonyfast         3       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for ipython in the last 31 days',
    width: 1000
  }), Chart({
    data:                username  authored authorAssociation
    9             choldgraf        11            MEMBER
    26             lresende         6            MEMBER
    24          kevin-bates         6            MEMBER
    34            takluyver         4            MEMBER
    32              parente         4            MEMBER
    6       alexarchambault         4            MEMBER
    39            yuvipanda         3            MEMBER
    37              winni2k         2       CONTRIBUTOR
    28                minrk         2            MEMBER
    20             jhamrick         2            MEMBER
    5             akchinSTC         2      COLLABORATOR
    10           clayadavis         1       CONTRIBUTOR
    11         craig-willis         1       CONTRIBUTOR
    38            xhumanoid         1       CONTRIBUTOR
    2              JarnoRFB         1       CONTRIBUTOR
    36             willingc         1            MEMBER
    35           wgcostello         1       CONTRIBUTOR
    3           RomanKornev         1       CONTRIBUTOR
    33              rkdarst         1       CONTRIBUTOR
    4                afshin         1            MEMBER
    31            nyanshell         1       CONTRIBUTOR
    30              mwcraig         1       CONTRIBUTOR
    29              mlucool         1       CONTRIBUTOR
    27               mgeier         1       CONTRIBUTOR
    7             benjaminr         1       CONTRIBUTOR
    25               lheagy         1      COLLABORATOR
    8               bskarda         1       CONTRIBUTOR
    23      katrinleinweber         1       CONTRIBUTOR
    22                jtpio         1       CONTRIBUTOR
    21           joergbrech         1       CONTRIBUTOR
    1              JMerrill         1       CONTRIBUTOR
    19                 heoa         1       CONTRIBUTOR
    18                 gaow         1       CONTRIBUTOR
    17             fresende         1       CONTRIBUTOR
    16                flixr         1       CONTRIBUTOR
    15              fisache         1       CONTRIBUTOR
    14  durwasa-chakraborty         1       CONTRIBUTOR
    13          damianavila         1            MEMBER
    12                csoja         1       CONTRIBUTOR
    0        GladysNalvarte         1       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
    

In [18]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:        org     repo username  closed
    0  ipython  ipython  Carreau      27,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 31 days',
    width: 1000
  }), Chart({
    data:         org                        repo     username  closed
    1   jupyter  cookiecutter-docker-stacks      parente       1
    2   jupyter               docker-stacks      parente       6
    3   jupyter          enterprise_gateway  kevin-bates       5
    4   jupyter          enterprise_gateway     lresende      12
    5   jupyter                     jupyter      jzf2101       2
    6   jupyter                jupyter-book    choldgraf      13
    7   jupyter           jupyter.github.io       afshin       1
    8   jupyter           jupyter.github.io      parente       1
    9   jupyter           jupyter.github.io     willingc       1
    10  jupyter                    jvm-repr       rdblue       4
    11  jupyter                   nbconvert    takluyver       3
    12  jupyter          nbconvert-examples      parente       1
    13  jupyter                    nbgrader     jhamrick       2
    14  jupyter                    nbviewer      parente       5
    15  jupyter                    notebook    takluyver       7
    16  jupyter                 repo2docker      betatim       3
    17  jupyter                 repo2docker    choldgraf       1
    18  jupyter                 repo2docker        minrk       2
    19  jupyter                 repo2docker    yuvipanda       4
    20  jupyter                     surveys      parente       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter in the last 31 days',
    width: 1000
  }), Chart({
    data:                 org        repo     username  closed
    21  jupyter-widgets  ipyleaflet  martinRenou      32,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter-widgets in the last 31 days',
    width: 1000
  }), Chart({
    data:            org                     repo      username  closed
    22  jupyterhub     alabaster-jupyterhub       betatim       2
    23  jupyterhub     alabaster-jupyterhub     choldgraf       2
    24  jupyterhub                   binder     choldgraf       2
    25  jupyterhub                binderhub       betatim       4
    26  jupyterhub                binderhub  captainsafia       2
    27  jupyterhub                binderhub         minrk      10
    28  jupyterhub                binderhub      willingc       4
    29  jupyterhub                binderhub     yuvipanda       2
    30  jupyterhub  configurable-http-proxy         minrk       2
    31  jupyterhub    firstuseauthenticator     yuvipanda       2
    32  jupyterhub               helm-chart     yuvipanda       2
    33  jupyterhub   jupyter-rsession-proxy    ryanlovett      14
    34  jupyterhub     jupyter-server-proxy    ryanlovett      26
    35  jupyterhub     jupyter-server-proxy     yuvipanda       8
    36  jupyterhub               jupyterhub         minrk      10
    37  jupyterhub              kubespawner  consideRatio       2
    38  jupyterhub      mybinder.org-deploy       betatim       4
    39  jupyterhub      mybinder.org-deploy  captainsafia       2
    40  jupyterhub      mybinder.org-deploy     choldgraf       4
    41  jupyterhub      mybinder.org-deploy  consideRatio       2
    42  jupyterhub      mybinder.org-deploy         minrk       6
    43  jupyterhub      mybinder.org-deploy    

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                         repo  number  \
  0           ipython                    ipykernel      12   
  1           ipython                        ipynb       6   
  2           ipython                  ipyparallel       3   
  3           ipython                      ipython      30   
  4           jupyter           enterprise_gateway       1   
  5           jupyter                         help      23   
  6           jupyter                      jupyter       1   
  7           jupyter                     notebook      25   
  8           jupyter                    qtconsole       1   
  9   jupyter-widgets                   ipyleaflet      24   
  10  jupyter-widgets                   ipywidgets      40   
  11  jupyter-widgets           jupyterlab-sidecar       4   
  12  jupyter-widgets                 midicontrols       4   
  13  jupyter-widgets                    pythreejs      16   
  14  jupyter-widgets       widget-ts-cookiecutter       4   
  15       jupyterhub                       binder       2   
  16       jupyterhub                    binderhub       8   
  17       jupyterhub                dockerspawner       2   
  18       jupyterhub                   helm-chart       2   
  19       jupyterhub         jupyter-server-proxy       2   
  20       jupyterhub                   jupyterhub      11   
  21       jupyterhub            ldapauthenticator       2   
  22       jupyterhub          mybinder.org-deploy       6   
  23       jupyterhub          nativeauthenticator       2   
  24       jupyterhub               oauthenticator       2   
  25       jupyterhub                 team-compass       2   
  26       jupyterhub       zero-to-jupyterhub-k8s       8   
  27       jupyterlab            jupyter-renderers      10   
  28       jupyterlab                   jupyterlab      85   
  29       jupyterlab        jupyterlab-commenting      15   
  30       jupyterlab             jupyterlab-latex       5   
  31       jupyterlab  jupyterlab-metadata-service       5   
  32       jupyterlab               jupyterlab-toc      10   
  33          nteract                      nteract      15   
  34          nteract                   nteract.io       1   
  35          nteract                         play       1   
  36          nteract                     semiotic       3   
  
                                    org/repo  
  0                        ipython/ipykernel  
  1                            ipython/ipynb  
  2                      ipython/ipyparallel  
  3                          ipython/ipython  
  4               jupyter/enterprise_gateway  
  5                             jupyter/help  
  6                          jupyter/jupyter  
  7                         jupyter/notebook  
  8                        jupyter/qtconsole  
  9               jupyter-widgets/ipyleaflet  
  10              jupyter-widgets/ipywidgets  
  11      jupyter-widgets/jupyterlab-sidecar  
  12            jupyter-widgets/midicontrols  
  13               jupyter-widgets/pythreejs  
  14  jupyter-widgets/widget-ts-cookiecutter  
  15                       jupyterhub/binder  
  16                    jupyterhub/binderhub  
  17                jupyterhub/dockerspawner  
  18                   jupyterhub/helm-chart  
  19         jupyterhub/jupyter-server-proxy  
  20                   jupyterhub/jupyterhub  
  21            jupyterhub/ldapauthenticator  
  22          jupyterhub/mybinder.org-deploy  
  23          jupyterhub/nativeauthenticator  
  24               jupyterhub/oauthenticator  
  25                 jupyterhub/team-compass  
  26       jupyterhub/zero-to-jupyterhub-k8s  
  27            jupyterlab/jupyter-renderers  
  28                   jupyterlab/jupyterlab  
  29        jupyterlab/jupyterlab-commenting  
  30             jupyterlab/jupyterlab-latex  
  31  jupyterlab/jupyterlab-metadata-service  
  32               jupyterlab/jupyterlab-toc  
  33                         

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                        repo  number  \
  0           ipython                     ipython      21   
  1           ipython                   traitlets       6   
  2           jupyter  cookiecutter-docker-stacks       2   
  3           jupyter               docker-stacks       4   
  4           jupyter          enterprise_gateway      10   
  5           jupyter                        help       4   
  6           jupyter                     jupyter       1   
  7           jupyter                jupyter-book       9   
  8           jupyter               jupyter-drive       1   
  9           jupyter              kernel_gateway       1   
  10          jupyter                   nbconvert       3   
  11          jupyter                      nbdime       2   
  12          jupyter                    nbgrader       1   
  13          jupyter                    notebook      24   
  14          jupyter                   qtconsole       1   
  15          jupyter                 repo2docker      10   
  16  jupyter-widgets                  ipyleaflet      32   
  17  jupyter-widgets                  ipywidgets       8   
  18       jupyterhub                      binder       2   
  19       jupyterhub                   binderhub      12   
  20       jupyterhub     configurable-http-proxy       2   
  21       jupyterhub       firstuseauthenticator       4   
  22       jupyterhub      jupyter-rsession-proxy       4   
  23       jupyterhub        jupyter-server-proxy      12   
  24       jupyterhub                  jupyterhub       6   
  25       jupyterhub  jupyterhub-deploy-teaching       2   
  26       jupyterhub              jupyterlab-hub       2   
  27       jupyterhub         nativeauthenticator      26   
  28       jupyterhub                team-compass       8   
  29       jupyterhub     the-littlest-jupyterhub       4   
  30       jupyterhub               traefik-proxy       8   
  31       jupyterhub      zero-to-jupyterhub-k8s      34   
  32       jupyterlab           jupyter-renderers       5   
  33       jupyterlab                  jupyterlab     173   
  34       jupyterlab     jupyterlab-google-drive      15   
  35       jupyterlab              jupyterlab-toc      20   
  36          nteract                   bookstore       4   
  37          nteract                    commuter       1   
  38          nteract                    hydrogen       3   
  39          nteract                  nteract.io       1   
  40          nteract                   papermill      13   
  41          nteract                    semiotic       6   
  
                                   org/repo  
  0                         ipython/ipython  
  1                       ipython/traitlets  
  2      jupyter/cookiecutter-docker-stacks  
  3                   jupyter/docker-stacks  
  4              jupyter/enterprise_gateway  
  5                            jupyter/help  
  6                         jupyter/jupyter  
  7                    jupyter/jupyter-book  
  8                   jupyter/jupyter-drive  
  9                  jupyter/kernel_gateway  
  10                      jupyter/nbconvert  
  11                         jupyter/nbdime  
  12                       jupyter/nbgrader  
  13                       jupyter/notebook  
  14                      jupyter/qtconsole  
  15                    jupyter/repo2docker  
  16             jupyter-widgets/ipyleaflet  
  17             jupyter-widgets/ipywidgets  
  18                      jupyterhub/binder  
  19                   jupyterhub/binderhub  
  20     jupyterhub/configurable-http-proxy  
  21       jupyterhub/firstuseauthenticator  
  22      jupyterhub/jupyter-rsession-proxy  
  23        jupyterhub/jupyter-server-proxy  
  24                  jupyterhub/jupyterhub  
  25  jupyterhub/jupyterhub-deploy-teaching  
  26              jupyterhub/jupyterlab-hub  
  27         jupyterhub/nativeauthenticator  
  28                jupyte

In [22]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [23]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/altair/utils/core.py:90: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  "Defaulting to nominal.".format(typ))


HConcatChart({
  hconcat: [Chart({
    data:             org                         repo     kind  count
    27   jupyterlab            jupyter-renderers  created   10.0
    28   jupyterlab                   jupyterlab  created   85.0
    29   jupyterlab        jupyterlab-commenting  created   15.0
    30   jupyterlab             jupyterlab-latex  created    5.0
    31   jupyterlab  jupyterlab-metadata-service  created    5.0
    32   jupyterlab               jupyterlab-toc  created   10.0
    54   jupyterlab      jupyterlab-google-drive  created    0.0
    86   jupyterlab            jupyter-renderers   closed    5.0
    87   jupyterlab                   jupyterlab   closed  173.0
    88   jupyterlab        jupyterlab-commenting   closed    0.0
    89   jupyterlab             jupyterlab-latex   closed    0.0
    90   jupyterlab  jupyterlab-metadata-service   closed    0.0
    91   jupyterlab               jupyterlab-toc   closed   20.0
    113  jupyterlab      jupyterlab-google-drive   closed   15.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for jupyterlab',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:                 org                    repo     kind  count
    9   jupyter-widgets              ipyleaflet  created   24.0
    10  jupyter-widgets              ipywidgets  created   40.0
    11  jupyter-widgets      jupyterlab-sidecar  created    4.0
    12  jupyter-widgets            midicontrols  created    4.0
    13  jupyter-widgets               pythreejs  created   16.0
    14  jupyter-widgets  widget-ts-cookiecutter  created    4.0
    68  jupyter-widgets              ipyleaflet   closed   32.0
    69  jupyter-widgets              ipywidgets   closed    8.0
    70  jupyter-widgets      jupyterlab-sidecar   closed    0.0
    71  jupyter-widgets            midicontrols   closed    0.0
    72  jupyter-widgets               pythreejs   closed    0.0
    73  jupyter-widgets  widget-ts-cookiecutter   closed    0.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for jupyter-widgets',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:         org         repo     kind  count
    0   ipython    ipykernel  created   12.0
    1   ipython        ipynb  created    6.0
    2   ipython  ipyparallel  created    3.0
    3   ipython      ipython  created   30.0
    37  ipython    traitlets  created    0.0
    59  ipython    ipykernel   closed    0.0
    60  ipython        ipynb   closed    0.0
    61  ipython  ipyparallel   closed    0.0
    62  ipython      ipython   closed   21.0
    96  ipython    traitlets   closed    6.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFon

In [24]:
# Set to datetime
for kind in ['createdAt', 'closedAt']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closedAt'] - closed['createdAt']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [25]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
    color='org',
)

Chart({
  data:                 org                        repo    time_open  \
  0           ipython                     ipython    18.652037   
  1           ipython                   traitlets   568.811424   
  2           jupyter  cookiecutter-docker-stacks    36.618924   
  3           jupyter               docker-stacks    16.190828   
  4           jupyter          enterprise_gateway    31.396458   
  5           jupyter                        help     0.160307   
  6           jupyter                     jupyter     1.284792   
  7           jupyter                jupyter-book    17.853819   
  8           jupyter               jupyter-drive  1181.490556   
  9           jupyter              kernel_gateway     1.053657   
  10          jupyter                   nbconvert    24.043310   
  11          jupyter                      nbdime     2.668154   
  12          jupyter                    nbgrader    73.154028   
  13          jupyter                    notebook    12.960943   
  14          jupyter                   qtconsole     0.234039   
  15          jupyter                 repo2docker     1.427124   
  16  jupyter-widgets                  ipyleaflet     6.196238   
  17  jupyter-widgets                  ipywidgets    13.684062   
  18       jupyterhub                      binder     0.218102   
  19       jupyterhub                   binderhub    31.665810   
  20       jupyterhub     configurable-http-proxy   509.415787   
  21       jupyterhub       firstuseauthenticator     3.203177   
  22       jupyterhub      jupyter-rsession-proxy     6.126684   
  23       jupyterhub        jupyter-server-proxy     6.139520   
  24       jupyterhub                  jupyterhub     0.559734   
  25       jupyterhub  jupyterhub-deploy-teaching   589.675150   
  26       jupyterhub              jupyterlab-hub   312.035937   
  27       jupyterhub         nativeauthenticator     9.950301   
  28       jupyterhub                team-compass    15.555093   
  29       jupyterhub     the-littlest-jupyterhub    32.174884   
  30       jupyterhub               traefik-proxy    28.486742   
  31       jupyterhub      zero-to-jupyterhub-k8s     3.351644   
  32       jupyterlab           jupyter-renderers     0.964826   
  33       jupyterlab                  jupyterlab    83.811331   
  34       jupyterlab     jupyterlab-google-drive    13.978056   
  35       jupyterlab              jupyterlab-toc    78.610365   
  36          nteract                   bookstore    25.910156   
  37          nteract                    commuter   405.695463   
  38          nteract                    hydrogen    91.341262   
  39          nteract                  nteract.io   637.332813   
  40          nteract                   papermill    30.204271   
  41          nteract                    semiotic     5.054873   
  
                                   org/repo  
  0                         ipython/ipython  
  1                       ipython/traitlets  
  2      jupyter/cookiecutter-docker-stacks  
  3                   jupyter/docker-stacks  
  4              jupyter/enterprise_gateway  
  5                            jupyter/help  
  6                         jupyter/jupyter  
  7                    jupyter/jupyter-book  
  8                   jupyter/jupyter-drive  
  9                  jupyter/kernel_gateway  
  10                      jupyter/nbconvert  
  11                         jupyter/nbdime  
  12                       jupyter/nbgrader  
  13                       jupyter/notebook  
  14                      jupyter/qtconsole  
  15                    jupyter/repo2docker  
  16             jupyter-widgets/ipyleaflet  
  17             jupyter-widgets/ipywidgets  
  18                      jupyterhub/binder  
  19                   jupyterhub/binderhub  
  20     jupyterhub/configurable-http-proxy  
  21       jupyterhub/firstuseauthenticator  
  22      jupyterhub/jupyter-rsession-proxy  
  23        jupyterhub/jupyter-server-pro

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [26]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'issue_id'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [27]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repo, i_issues in idata.groupby('repo'):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")

        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 31 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [28]:
commentors = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:     author  count authorAssociation
    0  betatim      2            MEMBER
    1  jhamman      2              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:              author  count authorAssociation
    2           Carreau    150            MEMBER
    58        takluyver     76            MEMBER
    55           stas00     45              NONE
    59        techtonik     33       CONTRIBUTOR
    5    LucianaMarques     24       CONTRIBUTOR
    30   kubaraczkowski     12              NONE
    32         memeplex     12       CONTRIBUTOR
    48         rmorshea      9      COLLABORATOR
    54            spott      9              NONE
    1           BoboTiG      9       CONTRIBUTOR
    56           stuaxo      9              NONE
    24          jakevdp      6              NONE
    22            ibell      6       CONTRIBUTOR
    50          sanyam5      6       CONTRIBUTOR
    16          dsblank      6       CONTRIBUTOR
    38          njsmith      6       CONTRIBUTOR
    34            minrk      6            MEMBER
    64           tomyun      6       CONTRIBUTOR
    13        billtubbs      6              NONE
    46           rgbkrk      3            MEMBER
    43         prittjam      3              NONE
    42       philippjfr      3              NONE
    44          pwm1234      3              NONE
    41         pganssle      3       CONTRIBUTOR
    45       rexendevar      3              NONE
    40          panaali      3              NONE
    0       ArtinSarraf      3              NONE
    47          rjurney      3              NONE
    49             rsvp      3              NONE
    51         shakedel      3              NONE
    39          noragen      3              NONE
    53        sixtyfive      3              NONE
    57        tacaswell      3       CONTRIBUTOR
    60       thadguidry      3              NONE
    61    thequackdaddy      3              NONE
    62         thinkjrs      3              NONE
    63           tomerk      3              NONE
    65          vmalloc      3              NONE
    52     sigmavirus24      3              NONE
    33     michael-pont      3              NONE
    37        neighthan      3              NONE
    17          esha-sg      3              NONE
    3       JafarShaik6      3              NONE
    4          LiranDan      3              NONE
    6         MehrdadDS      3              NONE
    7     SylvainCorlay      3            MEMBER
    8     TomAugspurger      3              NONE
    9           albertz      3              NONE
    10  ambareeshsrja16      3              NONE
    11      andrewarchi      3              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
    

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [30]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

In [31]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [32]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:     author  n_first_responses authorAssociation
    0  jhamman                  1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:               author  n_first_responses authorAssociation
    0            Carreau                 26            MEMBER
    37         takluyver                 12            MEMBER
    32          rmorshea                  3      COLLABORATOR
    20          memeplex                  2       CONTRIBUTOR
    33              rsvp                  1              NONE
    25         neighthan                  1              NONE
    26           noragen                  1              NONE
    27           panaali                  1              NONE
    28        philippjfr                  1              NONE
    29          prittjam                  1              NONE
    30           pwm1234                  1              NONE
    31           rjurney                  1              NONE
    34          shakedel                  1              NONE
    23  mohammedyunus009                  1              NONE
    35         sixtyfive                  1              NONE
    36            stas00                  1              NONE
    38         techtonik                  1       CONTRIBUTOR
    39     thequackdaddy                  1              NONE
    40          thinkjrs                  1              NONE
    41            tomerk                  1              NONE
    42            tomyun                  1       CONTRIBUTOR
    43           vmalloc                  1              NONE
    24         musicnerd                  1              NONE
    22             minrk                  1            MEMBER
    1        JafarShaik6                  1              NONE
    10           dsblank                  1       CONTRIBUTOR
    2           LiranDan                  1              NONE
    3     LucianaMarques                  1       CONTRIBUTOR
    4          MehrdadDS                  1              NONE
    5      SylvainCorlay                  1            MEMBER
    6            albertz                  1              NONE
    7        andrewarchi                  1              NONE
    8          billtubbs                  1              NONE
    9           cedriczg                  1              NONE
    11           esha-sg                  1              NONE
    21      michael-pont                  1              NONE
    12            fabrei                  1              NONE
    13          fianitnz                  1              NONE
    14            gitlio                  1              NONE
    15           jash101                  1              NONE
    16         jlstevens                  1       CONTRIBUTOR
    17            juckuc                  1              NONE
    18       jvanheugten                  1              NONE
    19          mazzma12                  1              NONE
    44         yuvipanda                  1      COLLABORATOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          rang

In [33]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>